In [1]:
import cv2
import torch
import numpy as np
from skimage import measure
import os
import pandas as pd

c:\Users\dangt\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = torch.hub.load('ultralytics/yolov5', 'custom', path='weights/best.pt')

Using cache found in C:\Users\dangt/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2023-3-20 Python-3.10.0 torch-1.13.1+cpu CPU

Fusing layers... 
Model summary: 157 layers, 7012822 parameters, 0 gradients, 15.8 GFLOPs
Adding AutoShape... 


In [4]:
# Load the image and resize it
image = cv2.imread("yellow_license_plate/8.jpg")
image = cv2.resize(image, (640, 640))

# Detect objects in the image
results = model(image)

for i, det in enumerate(results.xyxy[0]):
        # Lấy tọa độ bbox của vật thể thứ i
        bbox = det[0:4].cpu().numpy()

        # Cắt lấy vùng ảnh nằm trong bbox
        cropped_image = image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        cropped_image = cv2.resize(cropped_image, (256, 100))
        # Chuyển ảnh sang đen trắng
        gray = cv2.cvtColor(cropped_image, cv2.COLOR_BGR2GRAY)
        blurred = cv2.GaussianBlur(gray, (5, 5), 0)
        edges = cv2.Canny(blurred, threshold1=30, threshold2=100)
        edges = cv2.bitwise_not(edges)

        # Sử dụng hàm connectedComponents để tìm các vùng kết nối trên ảnh nhị phân
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
        num_labels, labels = cv2.connectedComponents(binary)

        # Sử dụng hàm connectedComponentsWithStats để tính toán diện tích của các vùng kết nối
        num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(binary)
        for i in range(num_labels):
            x, y, w, h, area = stats[i]
            if 90 < area < 1500:
                digit_img = binary[y:y+h, x:x+w]
                digit_img = cv2.resize(digit_img, (25,60))
                cv2.imshow("DIGIT", digit_img)
                cv2.waitKey()
                cv2.destroyAllWindows()

In [6]:
def readPlate(image, model):

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = model(image)

    for i, det in enumerate(results.xyxy[0]):
        # Lấy tọa độ bbox của vật thể thứ i
        bbox = det[0:4].cpu().numpy()

        # Cắt lấy vùng ảnh nằm trong bbox
        cropped_image = image[int(bbox[1]):int(bbox[3]), int(bbox[0]):int(bbox[2])]
        cropped_image = cv2.resize(cropped_image, (256, 100))
        return((bbox[2]-bbox[0])/(bbox[3]-bbox[1]))

In [13]:
folder_path = "car_long/" # đường dẫn đến folder chứa ảnh

ratio1 = []
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        # kiểm tra xem file có phải là ảnh jpg hoặc png không
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path) # đọc ảnh bằng OpenCV
        # sử dụng ảnh tại đây
        name, extension = os.path.splitext(filename) # Tách phần đuôi của tên tệp
        ratio1.append(readPlate(img, model))

In [14]:
folder_path = "yellow_license_plate/" # đường dẫn đến folder chứa ảnh

ratio = []
for filename in os.listdir(folder_path):
    if filename.endswith(".jpg") or filename.endswith(".png"): 
        # kiểm tra xem file có phải là ảnh jpg hoặc png không
        file_path = os.path.join(folder_path, filename)
        img = cv2.imread(file_path) # đọc ảnh bằng OpenCV
        # sử dụng ảnh tại đây
        name, extension = os.path.splitext(filename) # Tách phần đuôi của tên tệp
        ratio.append(readPlate(img, model))

In [15]:
print(ratio1)
smallest_ratio = min(ratio1)
print(smallest_ratio)

# # Tạo DataFrame từ mảng
# df = pd.DataFrame(pre_final, columns=['Image', 'Predicted'])

# # Xuất ra file CSV
# df.to_csv('predicted_output.csv', index=False)

2.4642847


In [16]:
print(ratio)
bigest_ratio = max(ratio)
print(bigest_ratio)


3.2667127
